# Rpy2 Test
Try this package out, it seems to be the most popular. Instead of calling entire R scripts from the python code this provides a pythonic way to access R packages which will make development easier

In [2]:
%reset -f

In [3]:
import rpy2
import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector

from rpy2.robjects import r, pandas2ri
pandas2ri.activate()

Note that in python script we will want to use a path based on file code was run from using something more like 
`dir_path = os.path.dirname(os.path.realpath(__file__))`

In [4]:
import os
cwd = os.getcwd()
data_dir = cwd.rsplit('/', 1)[0] +'/data'

In [5]:

data = robjects.r(f'read.csv("{data_dir}/HVS_data.csv", header=TRUE)')
data.head()

,recid,boro,uf1_1,uf1_3,uf1_4,uf1_5,uf1_6,uf1_7,uf1_8,uf1_9,...,fw74,fw75,fw76,fw77,fw78,fw79,fw80,il30per,il50per,il80per
1,1,1,9,9,9,1,9,9,9,9,...,37635401,33263895,10348898,11446240,32161257,9745805,13739972,21800,36250,58000
2,1,1,9,9,9,1,9,9,9,9,...,8270628,23614216,42797335,19594333,8525261,36330659,28940264,27200,45300,72500
3,1,1,9,9,9,1,9,9,9,9,...,8270628,23614216,42797335,19594333,8525261,36330659,28940264,24500,40800,65250
4,1,1,9,9,9,1,9,9,9,9,...,35934838,10027288,46175949,57167298,43401081,10664017,66729558,21800,36250,58000
5,1,1,9,9,9,1,9,9,9,9,...,50290497,10630692,32310018,70148673,42706530,10137630,64932695,21800,36250,58000


In [6]:
type(data)

pandas.core.frame.DataFrame

Ok sick could load in pandas from .pkl to see if they match?  Save that for later I think

For now do data cleaning in python instead of R

In [7]:
weights_correction = 10**5
data['fw'] = data['fw']/weights_correction

In [8]:
rep_weights_columns = [f'fw{i}' for i in range(1,81)]
data[rep_weights_columns] = data[rep_weights_columns]/weights_correction

Add occupancy final column

In [9]:
data['occ_final'] = data['recid']==1

Use survey package

In [10]:
survey_package = rpackages.importr('survey')
base = rpackages.importr('base')

In [11]:
survey_package = rpackages.importr('survey')
hu_design = survey_package.svrepdesign(variables=data.iloc[:, :188],
                          repweights=data[rep_weights_columns],
                          weights=data['fw'], combined_weights = True, type='other',
                         scale=4/80, rscales=1)

In [12]:
hu_design

<rpy2.robjects.vectors.ListVector object at 0x7fec18bbf680> [RTYPES.VECSXP]
R classes: ('svyrep.design',)
[StrSexpVe..., FloatSexp..., IntSexpVe..., NULLType, ..., FloatSexp..., ListSexpV..., FloatSexp..., BoolSexpV...]
  type: <class 'rpy2.robjects.vectors.StrVector'>
  <rpy2.robjects.vectors.StrVector object at 0x7fec1b4e4840> [RTYPES.STRSXP]
R classes: ('character',)
['other']
  scale: <class 'numpy.ndarray'>
  array([0.05])
  rscales: <class 'numpy.ndarray'>
  array([1], dtype=int32)
  rho: <class 'rpy2.rinterface_lib.sexp.NULLType'>
  <rpy2.rinterface_lib.sexp.NULLType object at 0x7fec2d897a40> [RTYPES.NILSXP]
...
  combined.weights: <class 'numpy.ndarray'>
  array([340.40517, 245.7711 , 245.7711 , ...,   5.30555,   5.26384,
         4.75251])
  variables: <class 'pandas.core.frame.DataFrame'>
               fw1        fw2        fw3        fw4        fw5        fw6  \
1      340.40517   95.03596  287.49468  372.60528  109.94249  339.16918   
2      245.77110  240.31593   70.29031  234.36272  436.09278   79.19871   
3      245.77110  240.31593   70.29031  234.36272  436.09278   79.19871   
4      370.98358  365.07874  366.30975  121.53024  389.18418  626.88738   
5      370.41707  392.22424  339.03791  120.56007  324.60931  713.16585   
...          ...        ...        ...        ...        ...        ...   
13262    4.75251    5.06389    7.50663    1.43235    4.79840    8.36583   
13263    4.73395    4.51096    8.52585    1.40724    4.81504    8.20003   
13264    5.30555    5.31645    9.32247    1.59172    5.45121    8.68934   
13265    5.26384    5.27781    5.43034    8.90737    1.64353    5.43560   
13266    4.75251    5.06389    4.39728    8.34836    1.40542    4.90059   

             fw7        fw8        fw9       fw10  ...       fw71       fw72  \
1      339.81330  130.22924   97.51346  354.29028  ...  106.38392  105.63807   
2      242.73862  386.19074  254.67778   68.64471  ...  249.13320  269.28284   
3      242.73862  386.19074  254.67778   68.64471  ...  249.13320  269.28284   
4      110.19894  361.58392  670.59417  359.74521  ...  588.09862  408.75175   
5      108.52205  349.25140  778.23475  366.16018  ...  567.52402  409.13360   
...          ...        ...        ...        ...  ...        ...        ...   
13262    4.29480    4.96644    4.55897    1.42082  ...    1.38368    8.35015   
13263    4.66800    4.89570    4.83707    1.38214  ...    1.40515    8.30783   
13264    5.27360    5.25558    5.29893    1.56828  ...    1.55459    9.29165   
13265    7.75374    5.77592    5.40679    5.22377  ...    8.48150    1.59465   
13266    7.33169    4.96644    4.55897    4.85100  ...    8.06466    1.43266   

            fw73       fw74       fw75       fw76       fw77       fw78  \
1       85.46191  376.35401  332.63895  103.48898  114.46240  321.61257   
2      212.16560   82.70628  236.14216  427.97335  195.94333   85.25261   
3      212.16560   82.70628  236.14216  427.97335  195.94333   85.25261   
4      405.10934  359.34838  100.27288  461.75949  571.67298  434.01081   
5      318.22810  502.90497  106.30692  323.10018  701.48673  427.06530   
...          ...        ...        ...        ...        ...        ...   
13262    5.08148    1.38516    7.40015    1.48790    4.90125    4.73834   
13263    4.70247    1.43110    8.20345    1.40781    4.61103    4.73872   
13264    5.36019    1.53107    9.20039    1.51857    5.40157    5.40400   
13265    8.31096    5.50802    1.63304    9.44963    1.37067    5.80245   
13266    8.67463    4.72922    1.26967    8.67211    1.43554    4.73834   

            fw79       fw80  
1       97.45805  137.39972  
2      363.30659  289.40264  
3      363.30659  289.40264  
4      106.64017  667.29558  
5      101.37630  649.32695  
...          ...        ...  
13262    4.69973    4.46682  
13263    4.92466    4.71949  
13264    5.16665    5.37879  
13265    5.20132    5.37014  
13266    4.69973    4.46682  

[13266 rows x 80 columns]
  pweights: <class 'numpy.ndarray'>
  

In [18]:
occ_subset = base.subset(hu_design, data['occ_final']==1)
survey_total = survey_package.svytotal(data['occ_final'], design=occ_subset)


In [19]:
survey_total

array([3109954.62348])

In [20]:
data[['occ_final']]

,occ_final
1,True
2,True
3,True
4,True
5,True
...,...
13262,True
13263,True
13264,True
13265,True


Ok correct total but no standard error. Close but not there yet. What happens when we group by sub borough area?

In [21]:
by_sub_borough_area = survey_package.svyby(formula=data['occ_final'], by=data[['boro','cd']], 
                                           design=hu_design, FUN=survey_package.svytotal)

Ok same measurements of total housing units as R but different standard errors. 

In [22]:
import numpy as np
np.isclose(by_sub_borough_area['V1'].sum(), survey_total[0])

True

Try and get SE by grouping by a column with single value. Jank but helpful to double check

In [23]:
data['a'] = 'a'

In [24]:
survey_total_with_se = survey_package.svyby(formula=data['occ_final'], by=data['a'], 
                                           design=hu_design, FUN=survey_package.svytotal)
survey_total_with_se

,by,V1,se
a,a,3.109955e+06,8611.514099


Matches page 47 of [official guide to calculating variances](https://www2.census.gov/programs-surveys/nychvs/technical-documentation/variances/2017-NYCHVS-Guide-to-Estimating-Variances.pdf), looks good

In [15]:
PUMS = robjects.r(f'read.csv("{data_dir}/PUMS_LEP_by_race.csv", header=TRUE)')
PUMS.head()

,person_id,PWGTP,RAC1P,HISP,NATIVITY,LANX,ENG,AGEP,PUMA,ST,...,PWGTP73,PWGTP74,PWGTP75,PWGTP76,PWGTP77,PWGTP78,PWGTP79,PWGTP80,LEP_by_race,a
1,20150000144861,15,White alone,Not Spanish/Hispanic/Latino,Foreign born,"No, speaks only English",N/A (less than 5 years old/speaks only English),24,4001,36,...,16,26,23,24,15,5,16,4,,True
2,20150000225401,15,White alone,Not Spanish/Hispanic/Latino,Native,"No, speaks only English",N/A (less than 5 years old/speaks only English),40,4001,36,...,14,14,4,15,24,5,15,4,,True
3,20150000267661,8,White alone,Not Spanish/Hispanic/Latino,Foreign born,"Yes, speaks another language",Well,83,4001,36,...,6,7,13,3,7,2,3,8,lep_wnh,True
4,20150000267662,8,White alone,Not Spanish/Hispanic/Latino,Foreign born,"Yes, speaks another language",Well,78,4001,36,...,7,9,13,2,8,2,3,8,lep_wnh,True
5,20150000268321,24,White alone,Not Spanish/Hispanic/Latino,Native,"No, speaks only English",N/A (less than 5 years old/speaks only English),26,4001,36,...,25,28,7,45,25,27,7,26,,True


In [14]:
rw_cols = [f"PWGTP{x}" for x in range(1, 81)]

In [21]:
PUMS['a'] =1

In [22]:
PUMS_design = survey_package.svrepdesign(variables=PUMS[['a']],
                          repweights=PUMS[rw_cols],
                          weights=PUMS['PWGTP'], combined_weights = True, type='other',
                         scale=4/80, rscales=1)

In [25]:
PUMS_by_PUMA = survey_package.svyby(formula=PUMS['a'], by=PUMS[['PUMA','LEP_by_race']], 
                                           design=PUMS_design, FUN=survey_package.svytotal)

In [26]:
PUMS_by_PUMA

,PUMA,LEP_by_race,V1,se
3701.,3701,,87022.0,1505.283553
3801.,3801,,142135.0,2495.568672
3901.,3901,,153956.0,1417.470079
4001.,4001,,131549.0,1942.002687
4101.,4101,,129240.0,1632.307959
3701.lep_anh,3701,lep_anh,1463.0,243.640596
3801.lep_anh,3801,lep_anh,1560.0,376.161593
3901.lep_anh,3901,lep_anh,2824.0,319.492988
4001.lep_anh,4001,lep_anh,4059.0,422.773459
4101.lep_anh,4101,lep_anh,9004.0,535.362249
